In [287]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import io
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
import string
from string import punctuation
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


processed=[]

#Read train data
file1 = open("/Users/poojithaamin/Desktop/DOCS/SJSU/SEM3/255/Assignment1/train.dat.txt")
lines = file1.readlines()

#Split train data into test and class
from sklearn.cross_validation import train_test_split
train, test = train_test_split(lines, train_size = 0.8, random_state=2)

#Read test data
file2 = open("/Users/poojithaamin/Desktop/DOCS/SJSU/SEM3/255/Assignment1/test.dat.txt")
lines2 = file2.readlines()
"""
#Remove numbers and punctuations
for x in lines2:
    filtered = filter(lambda c: not c.isdigit(), x)
    processed.append(filtered)
lines2 = processed

lines2 = [''.join(c for c in s if c not in string.punctuation) for s in lines2]
lines2 = [s for s in lines2 if s]
"""

#Split the train data into text and classes
train_cls = [l.split('\t')[0] for l in train]
train_docs = [l.split('\t')[1] for l in train]

"""
#Remove numbers and punctuations from train data set

processed=[]
for x in train_docs:
    filtered = filter(lambda c: not c.isdigit(), x)
    processed.append(filtered)
train_docs = processed


train_docs = [''.join(c for c in s if c not in string.punctuation) for s in train_docs]
train_docs = [s for s in train_docs if s]
"""

#Split the test data into text and classes
test_cls = [l.split('\t')[0] for l in test]
test_docs = [l.split('\t')[1] for l in test]

"""
#Remove numbers and punctuations from test data set
processed=[]
for x in test_docs:
    filtered = filter(lambda c: not c.isdigit(), x)
    processed.append(filtered)
test_docs = processed

test_docs = [''.join(c for c in s if c not in string.punctuation) for s in test_docs]
test_docs = [s for s in test_docs if s]

print train_cls[2]
print train_docs[2]
print test_cls[2]
print test_docs[2]
"""

#Split the complete train data into text and classes
cls = [l.split('\t')[0] for l in lines]
docs = [l.split('\t')[1] for l in lines]

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
"""
X_train_counts = count_vect.fit_transform(docs)
X_train_counts.shape
"""

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

"""
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

"""
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)



class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w.lower()) for w in analyzer(doc) if not w.isdigit() and len(w)>4])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojithaamin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [288]:
"""
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf_svm.fit(docs, cls)
predicted_svm = text_clf_svm.predict(lines2)
"""
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline 
from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import AdaBoostClassifier

text_mnb_stemmed = Pipeline([
                        ('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=0.0004, n_iter=6, random_state=42)),
 ])
    
"""
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', LinearSVC(penalty='l1', dual=False,
                                       tol=1e-2, class_weight='balanced')),
                            ])

"""
                            
text_mnb_stemmed = text_mnb_stemmed.fit(train_docs, train_cls)
predicted_mnb_stemmed = text_mnb_stemmed.predict(test_docs)

"""
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__alpha': (1e-2, 1e-3),
}
gs_clf_svm = GridSearchCV(text_mnb_stemmed, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(train_docs, train_cls)
predicted_mnb_stemmed = gs_clf_svm.predict(test_docs)
"""


/Users/poojithaamin/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


"\nfrom sklearn.model_selection import GridSearchCV\nparameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],\n              'tfidf__use_idf': (True, False),\n              'clf-svm__alpha': (1e-2, 1e-3),\n}\ngs_clf_svm = GridSearchCV(text_mnb_stemmed, parameters_svm, n_jobs=-1)\ngs_clf_svm = gs_clf_svm.fit(train_docs, train_cls)\npredicted_mnb_stemmed = gs_clf_svm.predict(test_docs)\n"

In [289]:
print(f1_score(predicted_mnb_stemmed, test_cls, average="macro"))
print(precision_score(predicted_mnb_stemmed, test_cls, average="macro"))
print(recall_score(predicted_mnb_stemmed, test_cls, average="macro")) 
print(classification_report_imbalanced(predicted_mnb_stemmed, test_cls))

0.614772601154
0.619182389943
0.619960190677
                   pre       rec       spe        f1       geo       iba       sup

          1       0.85      0.68      0.96      0.75      0.80      0.63       755
          2       0.50      0.56      0.95      0.53      0.73      0.51       248
          3       0.48      0.61      0.92      0.53      0.75      0.54       316
          4       0.79      0.70      0.93      0.74      0.81      0.64       766
          5       0.49      0.56      0.77      0.52      0.66      0.42       803

avg / total       0.66      0.63      0.89      0.64      0.75      0.55      2888



In [290]:
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=0.0004, n_iter=6, random_state=42)),
 ])

text_mnb_stemmed = text_mnb_stemmed.fit(docs, cls)
predicted_mnb_stemmed = text_mnb_stemmed.predict(lines2)


/Users/poojithaamin/anaconda/envs/py27/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [291]:
predicted_mnb_stemmed.tofile('/Users/poojithaamin/Desktop/DOCS/SJSU/SEM3/255/Assignment1/format.dat.txt', sep="\n", format="%s")
